

```
# This is formatted as code
```

# Style-GAN generator

Loads a pretrained Style-GAN2 generator in pytorch so that we can run tests for the geometric toolkit project.

This code is based on https://github.com/rosinality/stylegan2-pytorch and also the ILO code of Daras et al.

In [ ]:
#@title Run this cell to setup the repository (ignore the albumentations error)
# !git clone https://github.com/giannisdaras/ilo.git
# !mkdir ilo/files/inpainting
# !mkdir ilo/files/inversion
# !mkdir ilo/files/denoising
# !mkdir ilo/files/cls
!mkdir sample
!wget https://raw.githubusercontent.com/giannisdaras/ilo/f715ab73e8f62c7a9b1063cc133f0a1cdad093aa/requirements.txt
!pip install -r requirements.txt
# !wget https://zenodo.org/record/4536928/files/shape_predictor_68_face_landmarks.dat
!wget https://zenodo.org/record/4536928/files/stylegan2-ffhq-config-f.pt
# !mv stylegan2-ffhq-config-f.pt ilo/
from google.colab.patches import cv2_imshow
import cv2
import ipywidgets as widgets
import glob
import matplotlib.pyplot as plt
from ipywidgets import Layout
from PIL import Image
import numpy as np

In [ ]:
!git clone https://github.com/rosinality/stylegan2-pytorch.git

# Basic utilities: loading Obama, showing images, etc...

In [ ]:
from torchvision import utils
import torch
def show_img(currx):
  utils.save_image(
    currx,
    f"test.png",
    nrow=1,
    normalize=True,
    range=(-1, 1),
  )
  plt.imshow(cv2.imread('test.png')[:, :, ::-1])
  plt.show()
!wget https://pbs.twimg.com/media/EbCVjq6XYAEYr6Z.jpg
!mv EbCVjq6XYAEYr6Z.jpg obama.jpg
def load_obama():
  im = cv2.imread("obama.jpg")
  im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
  im = np.swapaxes(im,1,2)
  im = np.swapaxes(im,0,1)
  im = torch.from_numpy(im)
  im = torch.unsqueeze(im,0)
  im = im.float()
  im = 2 * im / 255
  im = im - 1
  return im
show_img(load_obama())

## Generate and show random images

In [ ]:
# This code loads the generator. It may take a couple of minutes to run.
import sys
sys.path.append('stylegan2-pytorch')
sys.argv = ['-f']
from model import Generator
import torch
import numpy as np
import argparse
from torchvision import utils
from generate import generate
# Code to load the generator is essentially taken from the code of https://github.com/rosinality/stylegan2-pytorch/blob/master/generate.py

device = "cuda"

parser = argparse.ArgumentParser(description="Generate samples from the generator")

parser.add_argument(
    "--size", type=int, default=1024, help="output image size of the generator"
)
parser.add_argument(
    "--sample",
    type=int,
    default=1,
    help="number of samples to be generated for each image",
)
parser.add_argument(
    "--pics", type=int, default=20, help="number of images to be generated"
)
parser.add_argument("--truncation", type=float, default=1, help="truncation ratio")
parser.add_argument(
    "--truncation_mean",
    type=int,
    default=4096,
    help="number of vectors to calculate mean for the truncation",
)
parser.add_argument(
    "--ckpt",
    type=str,
    default="stylegan2-ffhq-config-f.pt",
    help="path to the model checkpoint",
)
parser.add_argument(
    "--channel_multiplier",
    type=int,
    default=2,
    help="channel multiplier of the generator. config-f = 2, else = 1",
)

args = parser.parse_args()

args.latent = 512
args.n_mlp = 8

g_ema = Generator(
    args.size, args.latent, args.n_mlp, channel_multiplier=args.channel_multiplier
).to(device)
checkpoint = torch.load(args.ckpt)

g_ema.load_state_dict(checkpoint["g_ema"])

if args.truncation < 1:
    with torch.no_grad():
        mean_latent = g_ema.mean_latent(args.truncation_mean)
else:
    mean_latent = None

In [ ]:
def model_decode(model, latent, is_z=True):
  return model([latent], truncation=args.truncation, truncation_latent=mean_latent, input_is_latent=~is_z)[0]

In [ ]:
import torch.optim as optim

def get_graddir(model, z0,z1):
  x1 = model_decode(g_ema, z1).detach()
  optimizer = optim.Adam([z0], lr=0.001)
  z0.requires_grad = True
  x0 = model_decode(g_ema, z0)
  loss = torch.mean((x0 - x1)**2)
  loss.backward()
  grad_ret = torch.clone(z0.grad.detach())
  g_ema.zero_grad()
  return grad_ret

In [ ]:
## Histogram of dot products of nabla (g(z) - g(z1))^2|_{z = z0} for random z0, z1
latent_d = args.latent
numtrials = 1
dotprods = []
for i in range(100):
  print(i)
  z0 = torch.randn([numtrials, latent_d]).to(device)
  z1 = torch.randn([numtrials, latent_d]).to(device)
  graddir = get_graddir(g_ema, z0,z1)
  movedir = z1 - z0
  graddir = graddir / torch.norm(graddir)
  movedir = movedir / torch.norm(movedir)
  # print(graddir.shape)
  # print(movedir.shape)
  dotprods.extend([x for x in torch.sum(graddir * movedir,1).detach().cpu()])
plt.hist(np.asarray(dotprods))

In [ ]:
import torch.optim as optim

def get_loss(model, z0,z1):
  x1 = model_decode(g_ema, z1).detach()
  optimizer = optim.Adam([z0], lr=0.001)
  z0.requires_grad = True
  x0 = model_decode(g_ema, z0).detach()
  with torch.no_grad():
    loss = torch.mean((x0 - x1)**2)
    return loss

batch_size = 1
discretization = 50
dotprods = []
with torch.no_grad():
  for i in range(1):
    print(i)
    currlosses = []
    z0 = torch.randn([batch_size, latent_d]).to(device)
    z1 = torch.randn([batch_size, latent_d]).to(device)
    delta = z1 - z0
    for j in range(discretization):
      if j % 100 == 0:
        print(j)
      currz = z0 + delta * j / (discretization-1)
      currlosses.append(get_loss(g_ema,currz,z1))
    plt.plot(np.linspace(0,1,discretization),np.asarray(currlosses))
    plt.show()

In [ ]:
# Loss curve monotonicity during interpolation?

import torch.optim as optim
import math

def get_loss(model, z0,z1):
  x1 = model_decode(g_ema, z1).detach()
  optimizer = optim.Adam([z0], lr=0.01)
  z0.requires_grad = True
  x0 = model_decode(g_ema, z0).detach()
  with torch.no_grad():
    loss = torch.mean((x0 - x1)**2)
    return loss

batch_size = 1
discretization = 50
dotprods = []
with torch.no_grad():

  z1 = torch.randn([batch_size, latent_d]).to(device)
  for i in range(1):
    print(i)
    currlosses = []
    z0 = torch.randn([batch_size, latent_d]).to(device)
    delta = z1 - z0
    for j in range(discretization):
      t = j / (discretization-1)
      if j % 100 == 0:
        print(j)
      # currz = z0 * math.sqrt(1 - t**2) + z1 * t
      currz = z0 + t * delta
      currlosses.append(get_loss(g_ema,currz,z1))
    plt.plot(np.linspace(0,1,discretization),np.asarray(currlosses))
  plt.show()

In [ ]:
# Interpolating between two random images

import torch.nn.functional as F
import torch.optim as optim
import math

numtrials = 1
for trialnum in range(numtrials):
  latent_d = args.latent
  numsteps = 10000
  learning_rate = 0.0001
  z0 = torch.randn([numtrials, latent_d]).to(device)
  # z1 = torch.randn([numtrials, latent_d]).to(device)
  z1 = z0 * math.sqrt(0) + torch.randn([numtrials, latent_d]).to(device) * math.sqrt(1)

  ptlist = []
  currz = torch.clone(z0)

  currz.requires_grad = True
  x1 = model_decode(g_ema, z1).detach() # g_ema is the name of the generator
  utils.save_image(
    x1,
    f"test.png",
    nrow=1,
    normalize=True,
    range=(-1, 1),
  )
  plt.imshow(cv2.imread('test.png')[:, :, ::-1])
  plt.show()

  optimizer = optim.Adam([currz], lr=learning_rate)
  pt_list = []
  for step in range(numsteps):
    ptlist.append(torch.clone(currz))
    currx = model_decode(g_ema, currz)
    loss = torch.mean((currx - x1)**2)
    # if loss < 0.02:
    #   break

    loss.backward()
    # print(currz.grad)

    if step % 20 == 0:

      with torch.no_grad():
        print('step',step)
        print('g(z1) - g(z)',loss)
        print('z0 - z',torch.mean((currz - z0)**2))
        print('z1 - z',torch.mean((currz - z1)**2))
      utils.save_image(
          currx,
          f"test.png",
          nrow=1,
          normalize=True,
          range=(-1, 1),
      )
      plt.imshow(cv2.imread('test.png')[:, :, ::-1])
      plt.show()

    optimizer.step()
    optimizer.zero_grad()
  
    # with torch.no_grad():
    #   currz = currz * torch.norm(z1) / torch.norm(currz)
    # # print(torch.norm(currz.grad,dim=1))
    # # print(currz.grad)
    # with torch.no_grad():
    #   graddir = currz.grad / torch.norm(currz.grad, dim=1).view(numtrials, 1)
    #   currz.add_(-graddir, alpha=step_size)

      # print(graddir)


    # z0grad = z0grad / torch.norm(z0grad,dim=1).view((numtrials,1))
    # print(currz)

In [ ]:
torch.norm(z0)

In [ ]:
torch.norm(z1)

In [ ]:
delta = z1 - z0
deltac = currz - z0

In [ ]:
deltau = delta / torch.norm(delta)
deltacu = deltac / torch.norm(deltac)

In [ ]:
torch.sum(deltau * deltacu)

In [ ]:
torch.norm(currz)

In [ ]:
# Interpolating between two random images

import torch.nn.functional as F
import torch.optim as optim
import math

numtrials = 1
for trialnum in range(numtrials):
  latent_d = args.latent
  numsteps = 100000
  learning_rate = 0.0001
  # z0 = torch.randn([numtrials, latent_d]).to(device)
  # # z1 = torch.randn([numtrials, latent_d]).to(device)
  # z1 = z0 * math.sqrt(0) + torch.randn([numtrials, latent_d]).to(device) * math.sqrt(1)

  ptlist = []
  # currz = torch.clone(z0)
  currz = torch.clone(oldcurrz.detach())

  currz.requires_grad = True
  x1 = model_decode(g_ema, z1).detach() # g_ema is the name of the generator
  utils.save_image(
    x1,
    f"test.png",
    nrow=1,
    normalize=True,
    range=(-1, 1),
  )
  plt.imshow(cv2.imread('test.png')[:, :, ::-1])
  plt.show()

  optimizer = optim.Adam([currz], lr=learning_rate)
  pt_list = []
  for step in range(numsteps):
    ptlist.append(torch.clone(currz))
    currx = model_decode(g_ema, currz)
    loss = torch.mean((currx - x1)**2)
    # if loss < 0.02:
    #   break

    loss.backward()
    # print(currz.grad)

    if step % 20 == 0:

      with torch.no_grad():
        print('step',step)
        print('g(z1) - g(z)',loss)
        print('z0 - z',torch.mean((currz - z0)**2))
        print('z1 - z',torch.mean((currz - z1)**2))
      utils.save_image(
          currx,
          f"test.png",
          nrow=1,
          normalize=True,
          range=(-1, 1),
      )
      plt.imshow(cv2.imread('test.png')[:, :, ::-1])
      plt.show()

    optimizer.step()
    optimizer.zero_grad()
  
    # with torch.no_grad():
    #   currz = currz * torch.norm(z1) / torch.norm(currz)
    # # print(torch.norm(currz.grad,dim=1))
    # # print(currz.grad)
    # with torch.no_grad():
    #   graddir = currz.grad / torch.norm(currz.grad, dim=1).view(numtrials, 1)
    #   currz.add_(-graddir, alpha=step_size)

      # print(graddir)


    # z0grad = z0grad / torch.norm(z0grad,dim=1).view((numtrials,1))
    # print(currz)

In [ ]:
oldcurrz = torch.clone(currz)

In [ ]:
def show_img(currx):
  utils.save_image(
    currx,
    f"test.png",
    nrow=1,
    normalize=True,
    range=(-1, 1),
  )
  plt.imshow(cv2.imread('test.png')[:, :, ::-1])
  plt.show()

In [ ]:
import math
discretization = 10
with torch.no_grad():
  for i in range(discretization):
    t = i / ((discretization - 1))
    interpz = z0 * math.sqrt(1 - t**2) + z1 * t
    # interpz = currz * math.sqrt(1 - t**2) + z1 * t
    print(torch.mean((interpz - currz)**2))
    currx = model_decode(g_ema, interpz)
    show_img(currx)

In [ ]:
currz.shape

In [ ]:
plt.hist(np.asarray(currz.detach().cpu()))

In [ ]:
plt.hist(np.asarray(currz.detach().cpu()[0,:]),bins=50)

In [ ]:
plt.hist(np.asarray(z0.detach().cpu()[0,:]),bins=50)

**George:** Use Langevin diffusion to sample the level sets of the generator function g(z) = c. Even though g(z) is a stochastic function (some layers have added noise) the variance of g(z) should still be small so studying its level sets should be fine.

In [ ]:
# Interpolating between two random images

import torch.nn.functional as F
import torch.optim as optim
import math

numtrials = 1
for trialnum in range(numtrials):
  latent_d = args.latent
  numsteps = 10000
  learning_rate = 0.0001
  z0 = torch.randn([numtrials, latent_d]).to(device)
  # z1 = torch.randn([numtrials, latent_d]).to(device)
  z1 = z0 * math.sqrt(0) + torch.randn([numtrials, latent_d]).to(device) * math.sqrt(1)

  ptlist = []
  currz = torch.clone(z0)

  currz.requires_grad = True
  x1 = model_decode(g_ema, z1).detach() # g_ema is the name of the generator
  utils.save_image(
    x1,
    f"test.png",
    nrow=1,
    normalize=True,
    range=(-1, 1),
  )
  plt.imshow(cv2.imread('test.png')[:, :, ::-1])
  plt.show()

  optimizer = optim.Adam([currz], lr=learning_rate)
  pt_list = []
  for step in range(numsteps):
    ptlist.append(torch.clone(currz))
    currx = model_decode(g_ema, currz)
    loss = torch.mean((currx - x1)**2)
    # if loss < 0.02:
    #   break

    loss.backward()
    # print(currz.grad)

    if step % 20 == 0:

      with torch.no_grad():
        print('step',step)
        print('g(z1) - g(z)',loss)
        print('z0 - z',torch.mean((currz - z0)**2))
        print('z1 - z',torch.mean((currz - z1)**2))
      utils.save_image(
          currx,
          f"test.png",
          nrow=1,
          normalize=True,
          range=(-1, 1),
      )
      plt.imshow(cv2.imread('test.png')[:, :, ::-1])
      plt.show()

    optimizer.step()
    optimizer.zero_grad()

**George:** Study how well an epsilon box around randomly sampled z's in the latent space cover the space of [-1, 1]^{3x1024x1024} images.

In [ ]:
latent_d = args.latent
print(latent_d)

decode = lambda z : model_decode(g_ema, z).detach()

def compute_jacobian(f, z, h=1e-5):
  fz = f(z).squeeze()
  z = z.squeeze()
  n = fz.size()
  d = z.size()
  J = torch.zeros(tuple(n) + tuple(d))
  #for i in range(d):
  #  zh = z.copy()
  #  zh[i] += h
  #  fzh = f(zh).squeeze()
  #  J[:, i] = (fzh - fz) / h
  
  return J

# First study how isotropic the Gram matrices of the Jacobians are at random points.
# This should tell us if there are a few directions of large change in the output image.
trials = 100
conds = []
lmaxs = []
lmins = []
for i in range(trials):
  print(i)
  z = torch.randn([1, latent_d]).to(device)
  Jg = compute_jacobian(decode, z).view(-1, latent_d)
  G = torch.matmul(Jg.t(), Jg)

  cond = torch.linalg.cond(G).item()
  lmax = torch.lobpcg(G)[0].item()
  lmin = torch.lobpcg(G, largest=False)[0].item()

  conds.append(cond)
  lmaxs.append(lmax)
  lmins.append(lmin)

#eps = 1e-6
#mean = torch.zeros([num_samples, latent_d]).to(device)
#zs = eps * (mean + torch.randn([num_samples, latent_d]).to(device) - 0.5)
#
#xs = model_decode(g_ema, zs).detach()
#
#
#print(xs.shape)
#x0 = xs[0, :, :, :]
#print(x0)
#utils.save_image(
#    x0,
#    f"test.png",
#    nrow=1,
#    normalize=True,
#    range=(-1, 1),
#  )
#plt.imshow(cv2.imread('test.png')[:, :, ::-1])
#plt.show()

In [ ]:
y = torch.zeros((3, 1024, 1024, 512))

In [ ]:
f = lambda z : with torch.no_grad(): model_decode(g_ema, z).detach()
latent_d = args.latent
z = torch.randn([1, latent_d]).to(device)
fz = f(z).squeeze()
z = z.squeeze()
n = fz.size()
d = z.size()
print(tuple(n) + tuple(d))
J = np.zeros(tuple(n) + tuple(d))

## Sampling random standard Gaussian latent w generates monsters

In [ ]:
latent_d = args.latent
w = torch.randn([1, latent_d]).to(device)
x = model_decode(g_ema, w).detach()

utils.save_image(
    x,
    f"test.png",
    nrow=1,
    normalize=True,
    range=(-1, 1),
  )
plt.imshow(cv2.imread('test.png')[:, :, ::-1])
plt.show()